# Tugas 5 : LDA Berita

In [303]:
import numpy as np
import pandas as pd
import pickle

In [304]:
df = pd.read_csv('https://raw.githubusercontent.com/wahyuarilsaputra/dataset/main/databerita.csv')

## Pre Processing

### Menghilangkan Data yang Kosong

In [305]:
df.isnull().sum()

Judul    1
Isi      1
Label    3
dtype: int64

In [306]:
df.dropna(inplace=True)

In [307]:
df = df.drop_duplicates(subset=['Judul', 'Isi'])
df

,Judul,Isi,Label
0,Mahfud Md Usai Putusan MKMK Diketok: Salam Hor...,"Menteri Koordinator Bidang Politik, Hukum, dan...",Politik
1,"Waketum: Puncak HUT Golkar Acara Tunggal, Tak ...",Partai Golkar akan menggelar puncak perayaan H...,Politik
2,Pertamina Bentuk Generasi Muda Peduli Lingkung...,Pertamina bersama Subholding Refining & Petroc...,Edukasi
3,FKG UI Gelar Seminar Kesehatan Gigi-Mulut untu...,Stunting masih menjadi salah satu masalah kese...,Edukasi
4,Warga Israel Demo Desak Netanyahu Dipenjara ka...,Warga Israel melakukan unjuk rasa di depan ked...,Politik
...,...,...,...
1222,Daftar 366 Caleg DPRD Kabupaten Luwu Timur Len...,"Komisi Pemilihan Umum (KPU) Luwu Timur, Sulawe...",Politik
1223,Tekuk Salzburg Jadi Permudah Jalan Inter ke Fa...,Inter Milan harus susah payah mengalahkan Red ...,Sport
1224,Mitsubishi Pajero Didapuk Jadi Mobil Bersejarah,SUV legendaris Mitsubishi Pajero generasi pert...,Sport
1225,"Cerita Hilangnya Gitar Milik Gitaris Mocca, Bu...",Gitaris Sheila On 7 Eross Candra sempat dikaba...,Edukasi


In [308]:
df['Isi'][2]

'Pertamina bersama Subholding Refining & Petrochemical, PT Kilang Pertamina Internasional (PT KPI) Unit Dumai menggelar program Tanggung Jawab Sosial dan Lingkungan (TJSL) yang bernama Sekolah Energi Berdikari (SEB). Program yang bertajuk \'Energizing Sustainable Community\' ini berlangsung di SMK Negeri 2 Dumai, (30/10).Tujuan diadakannya program ini yakni untuk mendukung keterlibatan sekolah dan generasi muda dalam keberlanjutan lingkungan (environmental sustainability), serta mendorong mereka untuk memberikan dampak yang lebih besar ke masyarakat sekitar melalui edukasi dan penggunaan energi terbarukan.VP Corporate Communication Pertamina Fadjar Djoko Santoso mengatakan Program SEB merupakan inovasi yang dijalankan dari program Desa Energi Berdikari untuk mewujudkan transisi energi dari tingkat sekolah atau pendidikan, serta memberikan pemahaman kepada generasi muda untuk lebih peduli isu-isu lingkungan.\r\r\rSCROLL TO CONTINUE WITH CONTENT\r"Sekolah Energi Berdikari menyasar Sekola

### Cleaning Data

In [309]:
import re, string

# Text Cleaning
def cleaning(text):
    # Menghapus tag HTML
    text = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});').sub('', str(text))

    # Mengubah seluruh teks menjadi huruf kecil
    text = text.lower()

    # Menghapus spasi pada teks
    text = text.strip()

    # Menghapus Tanda Baca, karakter spesial, and spasi ganda
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub("â½ïgoal", "", text)
    text = re.sub("SCROLL TO CONTINUE WITH CONTENT", "", text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)


    # Menghapus Nomor
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Mengubah text yang berisi 'nan' dengan whitespace agar nantinya dapat dihapus
    text = re.sub('nan', '', text)

    return text

In [310]:
df['Isi'] = df['Isi'].apply(lambda x: cleaning(x))

### Tokenisasi Data

In [311]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [312]:
df['isi_tokens'] = df['Isi'].apply(lambda x: word_tokenize(x))
df[["Isi", "isi_tokens"]].head()

,Isi,isi_tokens
0,menteri koordinator bidang politik hukum dan k...,"[menteri, koordinator, bidang, politik, hukum,..."
1,partai golkar akan menggelar puncak perayaan h...,"[partai, golkar, akan, menggelar, puncak, pera..."
2,pertamina bersama subholding refining petroche...,"[pertamina, bersama, subholding, refining, pet..."
3,stunting masih menjadi salah satu masalah kese...,"[stunting, masih, menjadi, salah, satu, masala..."
4,warga israel melakukan unjuk rasa di depan ked...,"[warga, israel, melakukan, unjuk, rasa, di, de..."


### StopWord Data

In [313]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [314]:
from nltk.corpus import stopwords
from itertools import chain

stop_words = set(chain(stopwords.words('indonesian')))
df['isi_tokens'] = df['isi_tokens'].apply(lambda x: [w for w in x if not w in stop_words])

In [315]:
df['Isi'] = df['isi_tokens'].apply(lambda tokens: ' '.join(tokens))

In [316]:
df.head()

,Judul,Isi,Label,isi_tokens
0,Mahfud Md Usai Putusan MKMK Diketok: Salam Hor...,menteri koordinator bidang politik hukum keama...,Politik,"[menteri, koordinator, bidang, politik, hukum,..."
1,"Waketum: Puncak HUT Golkar Acara Tunggal, Tak ...",partai golkar menggelar puncak perayaan hut pa...,Politik,"[partai, golkar, menggelar, puncak, perayaan, ..."
2,Pertamina Bentuk Generasi Muda Peduli Lingkung...,pertamina subholding refining petrochemical pt...,Edukasi,"[pertamina, subholding, refining, petrochemica..."
3,FKG UI Gelar Seminar Kesehatan Gigi-Mulut untu...,stunting salah kesehatan fokus perhatian indon...,Edukasi,"[stunting, salah, kesehatan, fokus, perhatian,..."
4,Warga Israel Demo Desak Netanyahu Dipenjara ka...,warga israel unjuk kediaman perdana menteri pm...,Politik,"[warga, israel, unjuk, kediaman, perdana, ment..."


### Term Frekuensi

In [317]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
X_count = count_vectorizer.fit_transform(df['Isi'].values.astype('U'))

terms_count = count_vectorizer.get_feature_names_out()
df_countvect = pd.DataFrame(data = X_count.toarray(),columns = terms_count)
df_countvect

,aa,aaf,aalamiinjamaah,aamiiin,aamiin,aan,aanggota,aap,aarc,aarm,...,zumba,zuniasih,zunnun,zur,zusammenarbeit,zutobi,zuzanna,zwingli,zx,zyanawiah
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1221,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1222,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [318]:
with open("count_vectorizer_model.pkl", "wb") as file:
    pickle.dump(count_vectorizer, file)

## LDA Modeling

In [319]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD

### Modeling Data
- k=3
- alpha=0.1
- betha=0.2



In [320]:
lda_model = LatentDirichletAllocation(n_components=100, doc_topic_prior=0.2, topic_word_prior=0.1, random_state=42)
w1 = lda_model.fit_transform(df_countvect)
h1 = lda_model.components_

In [321]:
import pickle
with open("lda_model100.pkl", "wb") as file:
    pickle.dump(lda_model, file)

In [322]:
w1.shape

(1224, 100)

### proporsi topik pada dokumen

In [323]:
colnames = ["Topic" + str(i) for i in range(lda_model.n_components)]
docnames = ["Doc" + str(i) for i in range(len(df['Isi']))]
df_doc_topic = pd.DataFrame(np.round(w1,2),columns=colnames,index=docnames)
df_doc_topic['label'] = df['Label'].values

In [324]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic91,Topic92,Topic93,Topic94,Topic95,Topic96,Topic97,Topic98,Topic99,label
Doc0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.02,0.0,0.0,0.00,0.0,0.0,Politik
Doc1,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,Politik
Doc2,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,Edukasi
Doc3,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,Edukasi
Doc4,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,Politik
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc1219,0.0,0.0,0.0,0.0,0.66,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,Politik
Doc1220,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,Sport
Doc1221,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.16,0.0,0.0,0.00,0.0,0.0,Sport
Doc1222,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.65,0.0,0.0,Edukasi


In [325]:
topic_word_distributions = lda_model.components_

feature_names = count_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(topic_word_distributions):
    top_words_idx = topic.argsort()[::-1][:10]  # Ambil 10 kata teratas
    top_words = [feature_names[i] for i in top_words_idx]
    print(f"Topik {topic_idx}:")
    print(", ".join(top_words))
    print()

Topik 0:
dapil, sulawesi, selatan, makassar, pemilihan, andi, muhammad, muh, daerah, si

Topik 1:
motogp, mandalika, sirkuit, indonesia, pertamina, manchester, mu, event, sport, tim

Topik 2:
pengguna, little, henderson, ant, pulisic, lewis, fans, man, ivan, harry

Topik 3:
stroke, listrik, pln, tawuran, makassar, warga, pinus, twibbon, danny, masyarakat

Topik 4:
timur, kecamatan, towuti, nuha, iriana, mangkutana, tomoni, luwu, paud, wasuponda

Topik 5:
jabar, bey, gim, ombudsman, indonesia, gedung, acara, anies, relawan, pemerintah

Topik 6:
indonesia, olahraga, negara, atlet, pemuda, games, medali, ganefo, barat, nasional

Topik 7:
diabetes, diabetesi, indonesia, aprilia, gula, dukungan, sehat, penjualan, peci, kalori

Topik 8:
hewan, sedunia, santri, oktober, tanggal, peringatan, binatang, dunia, masyarakat, kekuasaan

Topik 9:
jakarta, air, kpu, makassar, pps, show, international, indonesia, dunia, bikestunt

Topik 10:
palestina, al, yahudi, stadion, orang, islam, israel, masjid, 

In [326]:
df_countvect['labels'] = df['Label'].values
df_countvect.shape

(1224, 31306)

In [327]:
from sklearn.model_selection import train_test_split
import time

X = df_doc_topic.drop('label', axis=1)
y = df_doc_topic['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Naive Bayes

In [328]:
from sklearn.naive_bayes import MultinomialNB
# from sklearn.naive_bayes import BernoulliNB
# from sklearn.naive_bayes import ComplementNB

In [329]:
start_time = time.time()
# naive_bayes_classifier = BernoulliNB(force_alpha=True)
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time
print("Waktu pelatihan model: {:.2f} detik".format(training_time))

Waktu pelatihan model: 0.02 detik


In [330]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = naive_bayes_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Akurasi:", accuracy)

print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred))

Akurasi: 0.7510204081632653
Laporan Klasifikasi:
              precision    recall  f1-score   support

     Edukasi       0.76      0.67      0.71        93
     Politik       0.76      0.79      0.77        78
       Sport       0.74      0.81      0.77        74

    accuracy                           0.75       245
   macro avg       0.75      0.76      0.75       245
weighted avg       0.75      0.75      0.75       245



In [331]:
with open("naive_bayes_model.pkl", "wb") as file:
    pickle.dump(naive_bayes_classifier, file)

### Support Vector Machine

In [332]:
from sklearn import svm

In [333]:
start_time = time.time()
clf = svm.SVC()
vector = clf.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time
print("Waktu pelatihan model: {:.2f} detik".format(training_time))

Waktu pelatihan model: 0.12 detik


In [334]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Akurasi:", accuracy)

print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred))

Akurasi: 0.7183673469387755
Laporan Klasifikasi:
              precision    recall  f1-score   support

     Edukasi       0.73      0.66      0.69        93
     Politik       0.74      0.71      0.72        78
       Sport       0.69      0.81      0.75        74

    accuracy                           0.72       245
   macro avg       0.72      0.72      0.72       245
weighted avg       0.72      0.72      0.72       245



### Decision Tree

In [335]:
from sklearn import tree

In [336]:
start_time = time.time()
clf = tree.DecisionTreeClassifier()
decision_tree = clf.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time
print("Waktu pelatihan model: {:.2f} detik".format(training_time))

Waktu pelatihan model: 0.03 detik


In [337]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred_clf = decision_tree.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_clf)
print("Akurasi:", accuracy)

print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred_clf))

Akurasi: 0.7142857142857143
Laporan Klasifikasi:
              precision    recall  f1-score   support

     Edukasi       0.70      0.69      0.70        93
     Politik       0.75      0.74      0.75        78
       Sport       0.69      0.72      0.70        74

    accuracy                           0.71       245
   macro avg       0.71      0.72      0.72       245
weighted avg       0.71      0.71      0.71       245



In [338]:
with open("tree_model.pkl", "wb") as file:
    pickle.dump(clf, file)

## Deteksi Data Baru

In [339]:
new_data = ['pertamina subholding refining petrochemical pt kilang pertamina internasional pt kpi unit dumai menggelar program tanggung sosial lingkungan tjsl bernama sekolah energi berdikari seb program bertajuk energizing sustainable community smk negeri dumai tujuan diadakannya program mendukung keterlibatan sekolah generasi muda keberlanjutan lingkungan environmental sustainability mendorong dampak masyarakat edukasi penggunaan energi terbarukan vp corporate communication pertamina fadjar djoko santoso program seb inovasi dijalankan program desa energi berdikari mewujudkan transisi energi tingkat sekolah pendidikan pemahaman generasi muda peduli isu isu lingkungan scroll to continue with content sekolah energi berdikari menyasar sekolah adiwiyata penggunaan energi terbarukan sekolah peaman pohon edukasi pembentukan kelompok ilmiah pecinta lingkungan keberlanjutan program tercapai tercipta generasi muda peduli lingkungan fajar keterangan tertulis selasa fajar program seb diharapkan mendorong tingkatan adiwiyata sekolah target dijalankan sekolah energi berdikari berjalan sekolah smkn dumai sekolah menghasilkan kwp energi listrik kebutuhan riset laboratorium setara penghematan biaya listrik rp juta berperan pengurangan emisi karbon kg co general manager gm pt kpi unit dumai didik subagyo program seb sejalan upaya pertamina mendukung kementerian lingkungan hidup kehuta klhk mengimplementasikan nilai nilai adiwiyata lembaga lembaga pendidikan program sejalan komitmen pertamina pengimplementasian environmental social and goverce esg sustainability development goals sdgs poin pendidikan berkualitas poin energi bersih terjangkau poin peggulangan perubahan iklim pilar program seb energizing sustainable community meliputi kegiatan edukasi sharing session perwira pertamina class inovasi pemasangan solar panel keterlibatan komunitas upcycling competition expo kesempatan walikota dumai diwakili asisten pemerintahan kesra yusrizal program seb dinilai berperan mendukung pemerintah menciptakan keberlanjutan generasi peduli lingkungan program mendukung pemanfaatan energi terbarukan komitmen perusahaan mengajak generasi muda memiliki pemahaman isu isu terkait lingkungan program sekolah adiwiyata pertamina instalasi solar panel pertamina rangkaian gelaran program seb meningkatkan pencapaian sekolah adiwiyata smkn dumai pelajar diedukasi terkait energi terbarukan diharapkan menciptakan lingkungan sekolah informasi kegiatan seb melibatkan pelajar smkn dumai kegiatan upcycling competition expo kegiatan pertamina mendorong pelajar berinovasi menciptakan alat semangat lingkungan kegiatan daur ulang gelaran kegiatan sharing session dibawakan staf khusus presiden bidang pendidikan inovasi ambassadors of sdg s indonesia billy mambrasar sharing session diisi perwira pt kpi unit dumai membawakan materi kelas kelas hadir kegiatan seb stakeholder pertamina dinas pendidikan kebudayaan kota dumai dinas lingkungan hidup kota dumai camat dumai selatan lurah bukit datuk tim pertamina hadir acara manager csr pt pertamina persero dian hapsari firasati tim sustainability thesa kemmy pratama pertamedika ihc dr sri muharni sarah']
df_baru = pd.DataFrame({'Isi': new_data})
df_baru['Isi'] = df_baru['Isi'].apply(lambda x: cleaning(x))
df_baru['Isi_tokens'] = df_baru['Isi'].apply(lambda x: word_tokenize(x))
df_baru['Isi_tokens'] = df_baru['Isi_tokens'].apply(lambda x: [w for w in x if not w in stop_words])
# df_baru['Isi_tokens'] = df_baru['Isi_tokens'].progress_apply(lambda x: stemmer.stem(' '.join(x)).split(' '))
df_baru['Isi'] = df_baru['Isi_tokens'].apply(lambda tokens: ' '.join(tokens))
df_baru[["Isi", "Isi_tokens"]].head()

,Isi,Isi_tokens
0,pertamina subholding refining petrochemical pt...,"[pertamina, subholding, refining, petrochemica..."


In [340]:
X_count_baru = count_vectorizer.transform(np.array(df_baru['Isi']))

terms_count = count_vectorizer.get_feature_names_out()
df_countvect_baru = pd.DataFrame(data = X_count_baru.toarray(),columns = terms_count)
df_countvect_baru

,aa,aaf,aalamiinjamaah,aamiiin,aamiin,aan,aanggota,aap,aarc,aarm,...,zumba,zuniasih,zunnun,zur,zusammenarbeit,zutobi,zuzanna,zwingli,zx,zyanawiah
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [341]:
w1_baru = lda_model.fit_transform(df_countvect_baru)
h1_baru = lda_model.components_

In [342]:
w1_baru.shape

(1, 100)

In [343]:
colnames = ["Topic" + str(i) for i in range(lda_model.n_components)]
docnames = ["Doc" + str(i) for i in range(len(df_baru['Isi']))]
df_doc_topic_baru = pd.DataFrame(np.round(w1_baru,2),columns=colnames,index=docnames)
df_doc_topic_baru

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic90,Topic91,Topic92,Topic93,Topic94,Topic95,Topic96,Topic97,Topic98,Topic99
Doc0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Prediksi Data

#### Data topic modeling

In [344]:
# y_pred_baru = naive_bayes_classifier.predict(df_doc_topic_baru)
# y_pred_baru = decision_tree.predict(df_doc_topic_baru)
y_pred_baru = clf.predict(df_doc_topic_baru)

y_pred_baru

array(['Sport'], dtype=object)